In [1]:
import sys
sys.path.append('../..')
from MPRA_predict.utils import *
from MPRA_predict.datasets import *

In [2]:
# bed_df = pd.read_csv('data/sequences.bed', sep='\t', header=None)
# bed_df.columns = ['chr', 'start', 'end', 'split']

# custom_order = {'train': 0, 'valid': 1, 'test': 2}
# bed_df = bed_df.sort_values(
#     by=['split', 'chr', 'start'],
#     key=lambda col: col.map(custom_order) if col.name == 'split' else col
# )

# # bed_df = bed_df.sort_values(by=['', 'chr', 'start'])
# bed_df.to_csv('data/enformer_data.csv')

In [3]:
bed_df = pd.read_csv('data/enformer_data.csv')
bed_df

,chr,start,end,split
0,chr18,928386,1059458,train
1,chr4,113630947,113762019,train
2,chr11,18427720,18558792,train
3,chr16,85805681,85936753,train
4,chr3,158386188,158517260,train
...,...,...,...,...
38166,chr19,33204702,33335774,test
38167,chr14,41861379,41992451,test
38168,chr19,30681544,30812616,test
38169,chr14,61473198,61604270,test


In [4]:
from pyfaidx import Fasta
genome = Fasta('../../../../genome/hg38.fa')

seqs = []
for row in tqdm(bed_df.itertuples(), total=len(bed_df)):
    seq = genome[row.chr][row.start:row.end].seq.upper()
    seqs.append(seq)
len(seqs)

100%|██████████| 38171/38171 [00:08<00:00, 4362.90it/s]


38171

In [5]:
count = 0
for seq in seqs:
    if seq == '':
        count += 1
print(count)

0


In [6]:
bed_df['seq'] = seqs
bed_df.to_csv('data/enformer_data_with_seq.csv', index=None)

In [3]:
bed_df = pd.read_csv('data/enformer_data_with_seq.csv')
bed_df

,chr,start,end,split,seq
0,chr18,928386,1059458,train,TGATTCTTAAGATTTTCCAGAAATGAGGATTCAGATACTCTTGCTT...
1,chr4,113630947,113762019,train,TAGCATATTAGCATTTTATGTAACATATCATGGATCACGACATATT...
2,chr11,18427720,18558792,train,AGGCCGGAATGCAGTGGCACAGTCTCGGCTCACTGCAACCTCCACC...
3,chr16,85805681,85936753,train,AGGGAGCTGCTGACCTTTGTGCCTGTAAATGGCTGTCCTCTCTGCC...
4,chr3,158386188,158517260,train,GCTGTTTTTTGTCTTTTAACTTTGTGGACATCATCTTTCCCTTTTA...
...,...,...,...,...,...
38166,chr19,33204702,33335774,test,CAGCCCCACCCCGCCAGGAGGCCTTCCGCCTCTGTGGCTCCGCCAT...
38167,chr14,41861379,41992451,test,TTTATATGCTTATTATATCATATTGACTGATGAATTCCTATGTTAT...
38168,chr19,30681544,30812616,test,GCTGGGGCACCAGCCAAGAAGCTGGATGGAGTCCTAAGGGCCCCTC...
38169,chr14,61473198,61604270,test,TTGTAAGAAGGGAAATGGTGGTGGTTGGCGGCGGGGGTAGGGGGGT...


In [4]:
df = bed_df[bed_df['split'] == 'test'].reset_index(drop=True)
df

,chr,start,end,split,seq
0,chr10,37555537,37686609,test,TATTTTTTTAAGTTGCCATTTGCTTGATGCTTAAGACCAAATGAAA...
1,chr14,87048845,87179917,test,TTCAAGTCCATGACTATAATTTCAAAGTGTGAGGTCTCTAATGATG...
2,chrX,136527085,136658157,test,GATCCCATCCTCTCCACCTGCTTTCTCCAGAATTTTATATTATTAG...
3,chr11,34042349,34173421,test,TGCGCACCTACAGGCCCAGCTACTTTGGAGCTTGAGGCAGGAGAAT...
4,chr19,10352757,10483829,test,GCACTCTATACACCGCCCCTTGCCTCCATAGGCCCCACCCCCGCAC...
...,...,...,...,...,...
1932,chr19,33204702,33335774,test,CAGCCCCACCCCGCCAGGAGGCCTTCCGCCTCTGTGGCTCCGCCAT...
1933,chr14,41861379,41992451,test,TTTATATGCTTATTATATCATATTGACTGATGAATTCCTATGTTAT...
1934,chr19,30681544,30812616,test,GCTGGGGCACCAGCCAAGAAGCTGGATGGAGTCCTAAGGGCCCCTC...
1935,chr14,61473198,61604270,test,TTGTAAGAAGGGAAATGGTGGTGGTTGGCGGCGGGGGTAGGGGGGT...


In [5]:
seqs = load_txt('data/enformer_seqs.txt')
len(seqs)

1937

In [6]:
dic = {seq : i for i, seq in enumerate(seqs)}

indices = np.array([dic[row.seq] for row in df.itertuples()])
indices

array([   0,    8,   16, ..., 1143, 1151, 1159])

In [7]:
targets = np.load('data/enformer_test_targets.npy')

targets = targets[indices]

In [9]:
np.save('data/enformer_test_targets.npy', targets)

In [ ]:
seq = seqs[0]

In [ ]:
from enformer_pytorch import from_pretrained

onehot = torch.tensor(str2onehot(pad_seq(seq, padded_length=196608), N_fill_value=0), dtype=torch.float32)
model_torch = from_pretrained('Enformer', use_tf_gamma=True).eval()
pred_torch = model_torch(onehot)
pred_torch = pred_torch['human'].detach().numpy()
torch.cuda.empty_cache()

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

onehot = tf.convert_to_tensor(str2onehot(pad_seq(seq, padded_length=393216), N_fill_value=0), dtype=tf.float32)
model_tf = hub.load("enformer_tfhub").model
pred_tf = model_tf.predict_on_batch(tf.expand_dims(onehot, axis=0))
pred_tf = pred_tf['human'][0].numpy()

# tf.keras.backend.clear_session()
# from tensorflow.python.framework import ops
# ops.reset_default_graph()
# from numba import cuda
# cuda.get_current_device().reset()
# import gc
# gc.collect()

In [ ]:
targets = np.load('data/enformer_targets.npy')

target = targets[0]

In [ ]:
print(pearsonr(pred_torch.reshape(-1), pred_tf.reshape(-1)))
print(pearsonr(pred_torch.reshape(-1), target.reshape(-1)))
print(pearsonr(pred_tf.reshape(-1), target.reshape(-1)))